In [7]:
#importing important 
import pandas as pd
import numpy as np
import os
import glob
import re

In [8]:
#function to chunk a given file in list form into 4 parts
def chunkin(f):
    chunk=[]
    #count the number of '' in a list and divide by 4
    #round it off
    find_index= round(f.count('')/4)
    #since rounding off will offset the slicing of the fourth in the coming steps, preserve the 3rd multiple 
    last_chunk = find_index*3
    #c is going to keep the count of '' in the list f
    c=0
    #j takes the value of i in later steps
    j=0
    #keeps the total count of '', similar to c but has a different function
    total_count=0
    #indexing all the elements in the list f
    for i in range(len(f)):
        #taking into account '' strings condition 
        if f[i] == '':
            #adds 1 to c
            c+=1
            #adds 1 to total_count
            total_count+=1
            #if c is equal to find_index (the 1/4th count of '' in the list f)
            if c == find_index:
                #bringing c back to 0 for the next round
                c=0
                #slicing the list into one part (the first time from [0:i], j=0) and appending to a list called 'chunk'
                chunk.append(f[j:i])
                #now assigning j with value of i
                j=i
                #this goes on for 2 more rounds where when c==find_index, the list is sliced further
                #but since the last chunck just needs the 3rd multiple as its starting point...
                #4th chunk checks if total_count has reached the last_chunk number to use the index j (since i is now assigned to j)
                #to  make the last slice
                if total_count==last_chunk:
                    #append last chunck to 'chunk'
                    chunk.append(f[j:])
    return chunk

In [9]:
#function to remove all empty strings from a list of strings
def half_clean(sub):
    sub_clean = []
    #taking chunk by chunk
    for x in sub:
        #considering elements that are not '' strings
        no_empty=[i for i in x if i]
        #appending list to a new list called 'sub_clean'
        sub_clean.append(no_empty)
    return sub_clean

In [10]:
#to remove all the time stamps and numberings that come before it to make a clean list (chunk) of strings.
#since the lists are chunks we need to preserve them together in a list called 'clean_chunks' before returning them.
#to reiterate: each 'clean_chunks' now contains a clean version of a subtitle text file, but it's divided into 4 parts. 
def pattern_del(list_):
    clean_chunks=[]
    #regex pattern for time stamps
    time_stamp='\d+:\d+\d+:\d+,\d+ --> \d+:\d+:\d+,\d+'
    #for sublist (chunk) in list (chunks)
    for x in list_:
        index=[]
        #indexing for each element in the list
        for i in range(len(x)-1):
            #formating the regex search according to index
            if re.findall('\d+', x[i]) and re.findall(time_stamp, x[i+1]):
                #saving the indexes i (numbering) and i+2 (time stamp) into a list called 'index'
                index.append(i)
                index.append(i+1)
        #converting chunk x into an array
        array=np.array(list(x))
        #using numpy operation to delete all the elements by their indexes which are present in the 'index' list
        new_array = np.delete(array, index)
        #converting new cleaned numpy array back to a list before appending to a new list
        clean_chunks.append(list(new_array))
    return clean_chunks

In [11]:
#path to the decompressed srt files
path = "data-20240401T135418Z-001/subtitle decompressed/*.srt"

target_directory = "data-20240401T135418Z-001/subtitle cleaned/"
os.makedirs(target_directory, exist_ok=True)

#iteratively accessing all srt files one by one
for filename in glob.glob(path, recursive=True):
    #next 3 steps are useful for the last part of this for loop in naming output files.
    #removing the path name 
    filename_ = filename.replace('data-20240401T135418Z-001/subtitle decompressed\\','')
    with open(filename, 'r', encoding='latin-1') as file:
        #read and split the data into a list by *line*
        f=file.read().splitlines()
        #splitting the list into 4 parts
        chunked_sub=chunkin(f)
        #removing empty strings from the 4 parts
        no_empty_element = half_clean(chunked_sub)
        #removing all the time stamps and numberings that come before it to clean all 4 parts.
        clean = pattern_del(no_empty_element)
        #colapsing the list of lists into one list
        full_text = [item for sublist in clean for item in sublist]
        #joining the strings/elements into one text string 
        text_join = '\n'.join(full_text)
        #just for good measure: encoding the data with 'utf-8'
        text = text_join.encode('utf-8')
        #opening a new text file in a new folder called 'subtitle cleaned' 
        #the text file will retain the name by its input file (filename_)
        with open(target_directory+filename_, "w", errors='ignore') as file:
            #write the'text' into the text file 
            file.write(text.decode('utf-8'))